In [ ]:
import logging, os
from datetime import datetime

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


In [ ]:
input_pdf = "../tests/llama2.pdf"


### 1. Using FitzPDFReader

In [ ]:
from fastrag.parser.llamaindex_readers.fitz_pdf_reader import FitzPDFReader

pdf_reader = FitzPDFReader(overlap=False)
documents = pdf_reader.load_data(file=input_pdf)


In [ ]:
print(len(documents))
documents[:2]

### 2. Using MarkDownReader

In [ ]:
# import pymupdf4llm
# llama_reader = pymupdf4llm.LlamaMarkdownReader()
# llama_docs = llama_reader.load_data(input_pdf)

In [ ]:
print(len(llama_docs))
llama_docs[:2]

### Add documents to the vector Index

In [ ]:
from fastrag.rag.retriever import BaseRetriever

In [ ]:

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import QdrantClient
from llama_index.core.schema import Document
from typing import Dict, Any, List


qdrant_url = "http://0.0.0.0:6333"
model_name = "BAAI/bge-m3"
qdrant_client = QdrantClient(url=qdrant_url, timeout=20)
EMBED_SIZE=512

embed_model = HuggingFaceEmbedding(model_name=model_name, max_length=EMBED_SIZE) # FIXME Changel max_length to consider high memory usage with bge-m3



In [ ]:
len(embed_model.get_query_embedding('test'))

In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP
model_url = 'https://huggingface.co/bartowski/Llama-3.2-3B-Instruct-GGUF/resolve/main/Llama-3.2-3B-Instruct-Q4_0.gguf'

llm = LlamaCPP(model_url=model_url, model_kwargs={"n_gpu_layers": 0}, verbose=True)


In [ ]:
llm.complete('who are you?')

In [ ]:
base_retriever = BaseRetriever(index_id='delete', embed_model=embed_model, qdrant_client=qdrant_client)

In [ ]:
# base_retriever.delete_index(index_id='delete')

In [ ]:
base_retriever.add_documents_to_index(documents=documents, index_id='delete')

In [ ]:
base_retriever.load_index()

In [ ]:
documents = base_retriever.dense_search("""We believe that the open release of LLMs, when done safely, will be a net benefit to society. Like all LLMs,
Llama 2 is a new technology that carries potential risks with use (Bender et al., 2021b; Weidinger et al., 2021;
Solaiman et al., 2023). """, dense_threshold=0.5)

print(len(documents))
for d in documents:
    print(d.score)